## 7.3 필기체를 생성하는 합성곱 계층 GAN 구현 
GAN을 이용해 필기체 숫자를 생성하는 인공신경망

In [1]:
# set to use CPU
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

### 7.3.1 공통 패키지 불러오기
- 공통 패키지 MNIST 데이터셋을 불러오는 케라스 서브패키지를 임포트

In [2]:
import numpy as np
from PIL import Image
import math
import os

from keras.datasets import mnist
from keras import models, layers, optimizers

# The Conv2D op currently only supports the NHWC tensor format on the CPU. The op was given the format: NCHW
import keras.backend as K
print(K.image_data_format())
# K.set_image_data_format('channels_first')
print(K.image_data_format())

channels_last
channels_last


### 7.3.2 사용자 정의 손실 함수 만들기
- 케라스가 제공하지 않는 함수를 사용자 정의로 만듬

In [3]:
import tensorflow as tf

def mse_4d(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true), axis=(1,2,3))

def mse_4d_tf(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_pred - y_true), axis=(1,2,3))

### 7.3.3 합성곱 계층 GAN 모델링
- GAN에 포함된 2가지 신경망인 생성망과 판별망을 모델링

In [22]:
class GAN(models.Sequential):
    def __init__(self, input_dim=32): # input_dim = args.n_train = 32
        """
        self, self.generator, self.discriminator are all models
        """
        super().__init__()
        self.input_dim = input_dim

        self.generator = self.GENERATOR()
        self.discriminator = self.DISCRIMINATOR()
        self.add(self.generator)
        self.discriminator.trainable = False
        self.add(self.discriminator)
        
        self.compile_all()

    def compile_all(self):
        # Compiling stage
        d_optim = optimizers.SGD(lr=0.0005, momentum=0.9, nesterov=True)
        g_optim = optimizers.SGD(lr=0.0005, momentum=0.9, nesterov=True)
        self.generator.compile(loss=mse_4d_tf, optimizer="SGD")
        self.compile(loss='binary_crossentropy', optimizer=g_optim)
        self.discriminator.trainable = True
        self.discriminator.compile(loss='binary_crossentropy', optimizer=d_optim)

    def GENERATOR(self):
        input_dim = self.input_dim

        model = models.Sequential()
        model.add(layers.Dense(1024, activation='tanh', input_dim=input_dim))
        model.add(layers.Dense(7 * 7 * 128, activation='tanh')) # H, W, C = 7, 7, 128
        model.add(layers.BatchNormalization())
        # The Conv2D op currently only supports the NHWC tensor format on the CPU.
        model.add(layers.Reshape((7, 7, 128), input_shape=(7 * 7 * 128,)))
        model.add(layers.UpSampling2D(size=(2, 2)))
        model.add(layers.Conv2D(64, (5, 5), padding='same', activation='tanh'))
        model.add(layers.UpSampling2D(size=(2, 2)))
        model.add(layers.Conv2D(1, (5, 5), padding='same', activation='tanh'))
        return model

    def DISCRIMINATOR(self):
        # The Conv2D op currently only supports the NHWC tensor format on the CPU. 
        model = models.Sequential()
        model.add(layers.Conv2D(64, (5, 5), padding='same', activation='tanh',
                                input_shape=(28, 28, 1)))
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))
        model.add(layers.Conv2D(128, (5, 5), activation='tanh'))
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))
        model.add(layers.Flatten())
        model.add(layers.Dense(1024, activation='tanh'))
        model.add(layers.Dense(1, activation='sigmoid'))
        return model

    def get_z(self, ln):
        input_dim = self.input_dim
        return np.random.uniform(-1, 1, (ln, input_dim))

    def train_both(self, x):
        ln = x.shape[0]
        # First trial for training discriminator
        z = self.get_z(ln)
        w = self.generator.predict(z, verbose=0)
        xw = np.concatenate((x, w))
        y2 = [1] * ln + [0] * ln
        y2 = np.array(y2).reshape(-1,1)
        
        #------------------
        # ValueError: logits and labels must have the same shape ((32, 1) vs ())
        # logits and labels must have the same shape ((16, 1) vs ())
        # (32, 28, 28, 1) 32
        print(xw.shape, y2.shape)
        #------------------
        d_loss = self.discriminator.train_on_batch(xw, y2)

        # Second trial for training generator
        z = self.get_z(ln)
        self.discriminator.trainable = False
        g_loss = self.train_on_batch(z, [1] * ln)
        self.discriminator.trainable = True

        return d_loss, g_loss

### 7.3.4 합성곱 계층 GAN 학습 시키기
- 앞서 만든 GAN의 모델링과 학습용 클래스를 사용해서 실제로 GAN을 학습하는 코드

In [23]:
################################
# GAN 학습하기
################################
def combine_images(generated_images):
    num = generated_images.shape[0]
    width = int(math.sqrt(num))
    height = int(math.ceil(float(num) / width))
    shape = generated_images.shape[2:]
    image = np.zeros((height * shape[0], width * shape[1]),
                     dtype=generated_images.dtype)
    for index, img in enumerate(generated_images):
        i = int(index / width)
        j = index % width
        image[i * shape[0]:(i + 1) * shape[0],
        j * shape[1]:(j + 1) * shape[1]] = img[0, :, :]
    return image

def get_x(X_train, index, BATCH_SIZE):
    return X_train[index * BATCH_SIZE:(index + 1) * BATCH_SIZE]

def save_images(generated_images, output_fold, epoch, index):
    image = combine_images(generated_images)
    image = image * 127.5 + 127.5
    Image.fromarray(image.astype(np.uint8)).save(
        output_fold + '/' +
        str(epoch) + "_" + str(index) + ".png")

def load_data(n_train):
    (X_train, y_train), (_, _) = mnist.load_data()
    return X_train[:n_train]

def train(args):
    BATCH_SIZE = args.batch_size
    epochs = args.epochs
    output_fold = args.output_fold
    input_dim = args.input_dim
    n_train = args.n_train

    os.makedirs(output_fold, exist_ok=True)
    print('Output_fold is', output_fold)

    X_train = load_data(n_train)

    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    # The Conv2D op currently only supports the NHWC tensor format on the CPU. The op was given the format: NCHW
    # X_train = X_train.reshape((X_train.shape[0], 1) + X_train.shape[1:]) # <-- NCHW format 
    X_train = X_train.reshape(X_train.shape + (1,)) # <-- NHWC format

    gan = GAN(input_dim)

    d_loss_ll = []
    g_loss_ll = []
    for epoch in range(epochs):
        print("Epoch is", epoch)
        print("Number of batches", int(X_train.shape[0] / BATCH_SIZE))

        d_loss_l = []
        g_loss_l = []
        for index in range(int(X_train.shape[0] / BATCH_SIZE)):
            x = get_x(X_train, index, BATCH_SIZE)

            d_loss, g_loss = gan.train_both(x)

            d_loss_l.append(d_loss)
            g_loss_l.append(g_loss)

        if epoch % 10 == 0 or epoch == epochs - 1:
            z = gan.get_z(x.shape[0])
            w = gan.generator.predict(z, verbose=0)
            save_images(w, output_fold, epoch, 0)

        d_loss_ll.append(d_loss_l)
        g_loss_ll.append(g_loss_l)

    gan.generator.save_weights(output_fold + '/' + 'generator', True)
    gan.discriminator.save_weights(output_fold + '/' + 'discriminator', True)

    np.savetxt(output_fold + '/' + 'd_loss', d_loss_ll)
    np.savetxt(output_fold + '/' + 'g_loss', g_loss_ll)

### 7.3.5 합성곱 계층 GAN 수행 
- 각 단계별 구현에 앞서 합성곱 계층 GAN의 수행 방법과 결과

In [24]:
def main():
    """
    # Not implemented in Notebook
    parser.add_argument('--batch_size', type=int, default=16,
        help='Batch size for the networks')
    parser.add_argument('--epochs', type=int, default=1000,
        help='Epochs for the networks')
    parser.add_argument('--output_fold', type=str, default='GAN_OUT',
        help='Output fold to save the results')
    parser.add_argument('--input_dim', type=int, default=10,
        help='Input dimension for the generator.')
    parser.add_argument('--n_train', type=int, default=32,
        help='The number of training data.')
    """
    class ARGS:
        def __init__(args):
            args.batch_size = 16
            args.epochs = 1000
            args.output_fold = 'GAN_OUT'
            args.input_dim = 10
            args.n_train = 32

    args = ARGS()
    train(args)
    
main()

Output_fold is GAN_OUT
Epoch is 0
Number of batches 2
(32, 28, 28, 1) (32, 1)


ValueError: in user code:

    /home/sjkim/anaconda3/envs/keras-gpu/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    /home/sjkim/anaconda3/envs/keras-gpu/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/sjkim/anaconda3/envs/keras-gpu/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/sjkim/anaconda3/envs/keras-gpu/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/sjkim/anaconda3/envs/keras-gpu/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:532 train_step  **
        loss = self.compiled_loss(
    /home/sjkim/anaconda3/envs/k

# 테스트 코드 (임시)

In [ ]:
n_train = 32
X_train = load_data(n_train)

X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = X_train.reshape(X_train.shape + (1,))
X_train.shape

In [42]:
import numpy as np
x = np.random.randn(32, 28, 28, 1)
x = x.astype(np.float32)

y = x
print(y.shape)
y = layers.Conv2D(64, (5, 5), padding='same', activation='tanh')(y)
print(y.shape)
y = layers.MaxPooling2D(pool_size=(2, 2))(y)
print(y.shape)
y = layers.Conv2D(128, (5, 5), activation='tanh')(y)
print(y.shape)
y = layers.MaxPooling2D(pool_size=(2, 2))(y)
print(y.shape)
y = layers.Flatten()(y)
print(y.shape)
y = layers.Dense(1024, activation='tanh')(y)
print(y.shape)
y = layers.Dense(1, activation='sigmoid')(y)
print(y.shape)

(32, 28, 28, 1)
(32, 28, 28, 64)
(32, 14, 14, 64)
(32, 10, 10, 128)
(32, 5, 5, 128)
(32, 3200)
(32, 1024)
(32, 1)


In [ ]:
model.add(layers.Conv2D(64, (5, 5), padding='same', activation='tanh',
                        input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(128, (5, 5), activation='tanh'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='tanh'))
model.add(layers.Dense(1, activation='sigmoid'))
return model

---
### 6.2.6 전체 코드

In [ ]:
################################
# 공통 패키지 불러오기
################################
import numpy as np
from PIL import Image
import math
import os

from keras import models, layers, optimizers
from keras.datasets import mnist
import keras.backend as K

K.set_image_data_format('channels_first')
print(K.image_data_format)

################################
# GAN 모델링
################################
class GAN(models.Sequential):
    def __init__(self, input_dim=64):
        """
        self, self.generator, self.discriminator are all models
        """
        super().__init__()
        self.input_dim = input_dim

        self.generator = self.GENERATOR()
        self.discriminator = self.DISCRIMINATOR()
        self.add(self.generator)
        self.discriminator.trainable = False
        self.add(self.discriminator)
        
        self.compile_all()

    def compile_all(self):
        # Compiling stage
        d_optim = optimizers.SGD(lr=0.0005, momentum=0.9, nesterov=True)
        g_optim = optimizers.SGD(lr=0.0005, momentum=0.9, nesterov=True)
        self.generator.compile(loss=mse_4d_tf, optimizer="SGD")
        self.compile(loss='binary_crossentropy', optimizer=g_optim)
        self.discriminator.trainable = True
        self.discriminator.compile(loss='binary_crossentropy', optimizer=d_optim)

    def GENERATOR(self):
        input_dim = self.input_dim

        model = models.Sequential()
        model.add(layers.Dense(1024, activation='tanh', input_dim=input_dim))
        model.add(layers.Dense(128 * 7 * 7, activation='tanh'))
        model.add(layers.BatchNormalization())
        model.add(layers.Reshape((128, 7, 7), input_shape=(128 * 7 * 7,)))
        model.add(layers.UpSampling2D(size=(2, 2)))
        model.add(layers.Conv2D(64, (5, 5), padding='same', activation='tanh'))
        model.add(layers.UpSampling2D(size=(2, 2)))
        model.add(layers.Conv2D(1, (5, 5), padding='same', activation='tanh'))
        return model

    def DISCRIMINATOR(self):
        model = models.Sequential()
        model.add(layers.Conv2D(64, (5, 5), padding='same', activation='tanh',
                                input_shape=(1, 28, 28)))
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))
        model.add(layers.Conv2D(128, (5, 5), activation='tanh'))
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))
        model.add(layers.Flatten())
        model.add(layers.Dense(1024, activation='tanh'))
        model.add(layers.Dense(1, activation='sigmoid'))
        return model

    def get_z(self, ln):
        input_dim = self.input_dim
        return np.random.uniform(-1, 1, (ln, input_dim))

    def train_both(self, x):
        ln = x.shape[0]
        # First trial for training discriminator
        z = self.get_z(ln)
        w = self.generator.predict(z, verbose=0)
        xw = np.concatenate((x, w))
        y2 = [1] * ln + [0] * ln
        d_loss = self.discriminator.train_on_batch(xw, y2)

        # Second trial for training generator
        z = self.get_z(ln)
        self.discriminator.trainable = False
        g_loss = self.train_on_batch(z, [1] * ln)
        self.discriminator.trainable = True

        return d_loss, g_loss

################################
# GAN 학습하기
################################
def combine_images(generated_images):
    num = generated_images.shape[0]
    width = int(math.sqrt(num))
    height = int(math.ceil(float(num) / width))
    shape = generated_images.shape[2:]
    image = np.zeros((height * shape[0], width * shape[1]),
                     dtype=generated_images.dtype)
    for index, img in enumerate(generated_images):
        i = int(index / width)
        j = index % width
        image[i * shape[0]:(i + 1) * shape[0],
        j * shape[1]:(j + 1) * shape[1]] = img[0, :, :]
    return image


def get_x(X_train, index, BATCH_SIZE):
    return X_train[index * BATCH_SIZE:(index + 1) * BATCH_SIZE]


def save_images(generated_images, output_fold, epoch, index):
    image = combine_images(generated_images)
    image = image * 127.5 + 127.5
    Image.fromarray(image.astype(np.uint8)).save(
        output_fold + '/' +
        str(epoch) + "_" + str(index) + ".png")


def load_data(n_train):
    (X_train, y_train), (_, _) = mnist.load_data()

    return X_train[:n_train]

def train(args):
    BATCH_SIZE = args.batch_size
    epochs = args.epochs
    output_fold = args.output_fold
    input_dim = args.input_dim
    n_train = args.n_train

    os.makedirs(output_fold, exist_ok=True)
    print('Output_fold is', output_fold)

    X_train = load_data(n_train)

    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    X_train = X_train.reshape((X_train.shape[0], 1) + X_train.shape[1:])

    gan = GAN(input_dim)

    d_loss_ll = []
    g_loss_ll = []
    for epoch in range(epochs):
        print("Epoch is", epoch)
        print("Number of batches", int(X_train.shape[0] / BATCH_SIZE))

        d_loss_l = []
        g_loss_l = []
        for index in range(int(X_train.shape[0] / BATCH_SIZE)):
            x = get_x(X_train, index, BATCH_SIZE)

            d_loss, g_loss = gan.train_both(x)

            d_loss_l.append(d_loss)
            g_loss_l.append(g_loss)

        if epoch % 10 == 0 or epoch == epochs - 1:
            z = gan.get_z(x.shape[0])
            w = gan.generator.predict(z, verbose=0)
            save_images(w, output_fold, epoch, 0)

        d_loss_ll.append(d_loss_l)
        g_loss_ll.append(g_loss_l)

    gan.generator.save_weights(output_fold + '/' + 'generator', True)
    gan.discriminator.save_weights(output_fold + '/' + 'discriminator', True)

    np.savetxt(output_fold + '/' + 'd_loss', d_loss_ll)
    np.savetxt(output_fold + '/' + 'g_loss', g_loss_ll)


################################
# GAN 예제 실행하기
################################
# import argparse

def main():
    """
    # Not implemented in Notebook
    parser.add_argument('--batch_size', type=int, default=16,
        help='Batch size for the networks')
    parser.add_argument('--epochs', type=int, default=1000,
        help='Epochs for the networks')
    parser.add_argument('--output_fold', type=str, default='GAN_OUT',
        help='Output fold to save the results')
    parser.add_argument('--input_dim', type=int, default=10,
        help='Input dimension for the generator.')
    parser.add_argument('--n_train', type=int, default=32,
        help='The number of training data.')
    """
    class ARGS:
        def __init__(args):
            args.batch_size = 16
            args.epochs = 1000
            args.output_fold = 'GAN_OUT'
            args.input_dim = 10
            args.n_train = 32

    args = ARGS()
    train(args)
    
main()